In [1]:
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.vocab import build_vocab_from_iterator

from konlpy.tag import Hannanum

from tqdm import tqdm
import pandas as pd

In [2]:
kor_tokenizer = get_tokenizer(Hannanum().morphs)
eng_tokenizer = get_tokenizer('spacy', language='en')

C:\Users\kmiii\.conda\envs\py37\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [3]:
pd.read_csv('./datasets/korNeng_corpus/train_data.csv')

,KOR,ENG
0,"해외에는 세계 최초이자 일본 최대 MCN 기업인 UUUM, 디즈니에 인수된 메이커 ...","Overseas, UUUM, the world's first and the larg..."
1,세계 최초의 노인 인권 전담 국제기구가 한국에 들어선다.,The world's first international organization d...
2,세계 최대 소셜미디어 페이스북 사용자 수억명의 계정 비밀번호(패스워드)가 암호화 장...,"""Account passwords (passwords) of hundreds of ..."
3,윤정헌씨에 대한 고문이 30여년 만에야 사실로 확인된 것은 이런 범죄가 얼마나 밝혀...,The fact that the torture of Yoon Jeong-heon w...
4,"동락연희단은 앉은 채로 연주했지만, 숨쉴틈 없이 몰아치는 소리에 관객들은 가만히 앉...",The Dongrakyeonhee group performed in a sittin...
...,...,...
1121687,검찰 측은 경찰 수사에 의문을 품고 보완수사를 요구하더라도 경찰이 “정당하지 못한 ...,The prosecution is questioning the police's in...
1121688,"문을 열자마자 벽에 걸린 진분홍빛 악어가죽이 눈에 들어오고, 진열장 안에서는 저마다...","As soon as you open the door, the pink crocodi..."
1121689,우리에게 겨우 일주일밖에 남지 않았습니다.,We only have a week left.
1121690,가서 시외버스만 정차하는 버스정류장을 찾으면 되지.,We can go find a bus stop only for intercity b...


In [4]:
def vocab_iterator(strings, tokenizer):
    for string_ in tqdm(strings):
        yield tokenizer(string_)

In [5]:
kor_vocab_base = pd.read_csv('./datasets/korNeng_corpus/train_data.csv')['KOR']
eng_vocab_base = pd.read_csv('./datasets/korNeng_corpus/train_data.csv')['ENG']

In [6]:
kor_vocab = build_vocab_from_iterator(vocab_iterator(kor_vocab_base, kor_tokenizer), specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=5)
eng_vocab = build_vocab_from_iterator(vocab_iterator(eng_vocab_base, eng_tokenizer), specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=5)

100%|██████████| 1121692/1121692 [00:44<00:00, 24938.89it/s]


In [11]:
kor_vocab.set_default_index(kor_vocab['<unk>'])
eng_vocab.set_default_index(eng_vocab['<unk>'])

In [12]:
torch.save(kor_vocab, './datasets/korNeng_corpus/kor_vocab.pt')
torch.save(eng_vocab, './datasets/korNeng_corpus/eng_vocab.pt')

In [13]:
def data_process(file_path):
    raw_kor_iter = iter(pd.read_csv(file_path)['KOR'])
    raw_eng_iter = iter(pd.read_csv(file_path)['ENG'])
    data = []
    for (raw_kor, raw_eng) in tqdm(zip(raw_kor_iter, raw_eng_iter)):
        kor_tensor_ = torch.tensor([kor_vocab[token] for token in kor_tokenizer(raw_kor)], dtype = torch.long)
        eng_tensor_ = torch.tensor([eng_vocab[token] for token in eng_tokenizer(raw_eng)], dtype = torch.long)

        data.append((kor_tensor_, eng_tensor_))

    return data


In [14]:
train_data = data_process('./datasets/korNeng_corpus/train_data.csv')
torch.save(train_data, './datasets/korNeng_corpus/train_tensor.pt')

884994it [49:41, 370.06it/s]

In [ ]:
valid_data = data_process('./datasets/korNeng_corpus/validation_data.csv')
torch.save(valid_data, './datasets/korNeng_corpus/valid_tensor.pt')

0it [00:00, ?it/s]


RuntimeError: Token 은성유치원은 not found and default index is not set

In [ ]:
test_data = data_process('./datasets/korNeng_corpus/test_data.csv')
torch.save(test_data, './datasets/korNeng_corpus/test_tensor.pt')

In [ ]:
!shutdown /h